In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import warnings
warnings.filterwarnings('ignore')

# LIBRARY IMPORT
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from scipy.stats import pearsonr
from mpl_toolkits.basemap import Basemap, cm

In [2]:
# Load intensification rate data
int_rate = pd.read_csv('Data/computed-data/computed_annual-intensification-rate_JTWC.csv')
int_rate = int_rate[int_rate['YEAR']>=1955]

# Load intensification duration data
int_dur = pd.read_csv('Data/computed-data/computed_annual-intensification-duration_JTWC.csv')
int_dur = int_rate[int_dur['YEAR']>=1955]

In [3]:
# Define function to find correlation
def find_corr_rate(data):
    return pearsonr(data['t_an'].values,int_rate['Intensification Rate'].values)[0]

In [8]:
sub_season_corr

<xarray.Dataset>
Dimensions:  (lat: 62, lon: 81)
Coordinates:
  * lat      (lat) float64 -0.5 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5 ...
  * lon      (lon) float64 99.5 100.5 101.5 102.5 103.5 104.5 105.5 106.5 ...
Data variables:
    corr     (lat, lon) float64 0.0 0.0 0.0 0.0 0.002069 0.004723 0.02185 ...

In [5]:
depths = [   0,   10,   20,   30,   50,   75,  100,  125,  150,  200,
        250,  300,  400,  500,  600,  700]
fig, ax = plt.subplots(nrows = 2, ncols = 2)
i=0
for depth in [0]:
    print(depth)
    # Load sub surface temperature data
    sub = pd.read_csv('Data/sub-surface-temperature/raw-data_{}m-sub-temp.csv'.format(depth),
                      parse_dates=['time'])
    sub = sub[['depth','lat','lon','t_mn','t_an','t_SE','time']]
    sub = sub.drop_duplicates()

    # Find seasonal mean for sub-surface temperature
    sub_season_mean = sub.groupby(['lat','lon','time'])['t_an'].mean().reset_index()
    
    sub_season_corr = sub_season_mean.groupby(['lat','lon']).apply(find_corr_rate).rename('corr')
    sub_season_corr = sub_season_corr.to_frame()
    sub_season_corr = xr.Dataset.from_dataframe(sub_season_corr)

    sub_season_corr = sub_season_corr.fillna(0)

    data = sub_season_corr.variables['corr'][:]
    lats = sub_season_corr.variables['lat'][:]
    lons = sub_season_corr.variables['lon'][:]

    # Width of the map used is set to be 1,000 km
    width = 1e6

    # Determine whether the cyclone move into the big box
    map = Basemap(projection='cyl',resolution='c',llcrnrlon=95,\
            llcrnrlat=-5,urcrnrlon=185,urcrnrlat=65, ax=ax[i])
    i+=1
    im1 = map.pcolormesh(lons, lats, data,cmap=plt.cm.jet,latlon=True)

    cb = map.colorbar(im1,"right", size="5%", pad="2%")

    # make image bigger:
    plt.title('Seasonal sub - Intensification Rate {}m Correlation Map'.format(depth),
              fontsize=16)
    plt.gcf().set_size_inches(10,10)
    map.drawcoastlines()
    map.drawmapboundary()
    map.fillcontinents()
    map.drawparallels(np.arange(-90.,91.,10.),labels=[True,False,False,True])
    map.drawmeridians(np.arange(-180.,181.,10.),labels=[True,False,False,True])

plt.show()

0


AttributeError: 'numpy.ndarray' object has no attribute 'ishold'